# Amplifierd API - Amplified Directories

This notebook demonstrates the amplified directories feature for managing multiple working directory contexts.

## Overview

Amplified directories enable the daemon to manage multiple working directory contexts, each with its own settings and configuration.

**Key Concepts**:
- **Amplified Directory**: Any directory within `AMPLIFIERD_DATA_PATH` containing a `.amplified/` marker
- **Root is Always Amplified**: The root directory is auto-amplified on daemon startup
- **Profile Inheritance**: Directories inherit `default_profile` from parent if not explicitly set
- **Session Association**: Sessions are tied to specific amplified directories (1 directory → 0..N sessions)

**Metadata Schema**:
```json
{
  "default_profile": "foundation/foundation",  // Required: Profile for new sessions
  "name": "My Project",                         // Optional: Human-readable name
  "description": "Project description",         // Optional: Description
  // ... other user-defined fields
}
```

**Directory Structure Example**:
```
AMPLIFIERD_DATA_PATH/
├── .amplified/              # Root (auto-created, foundation/foundation)
├── projects/
│   ├── app1/
│   │   └── .amplified/      # App 1 (inherits from root or explicit)
│   └── app2/
│       └── .amplified/      # App 2 (can have different profile)
```

In [ ]:
# Setup and helper functions
import json
from typing import Any

import requests

BASE_URL = "http://127.0.0.1:8421"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, label: str = "RESPONSE") -> Any:
    """Pretty-print API response with formatting."""
    print(f"\n{'=' * 60}")
    print(f"{label}")
    print(f"{'=' * 60}")
    print(f"Status: {response.status_code}")

    if response.status_code == 204:
        print("No content returned (successful operation)")
        return None

    try:
        data = response.json()
        print(json.dumps(data, indent=2))
        return data
    except Exception as e:
        print(f"Error parsing response: {e}")
        print(f"Raw response: {response.text}")
        return None


print("✓ Setup complete")
print(f"  API Base: {API_BASE}")

## List Amplified Directories

Start by listing all amplified directories. The root directory should already be amplified (auto-created on daemon startup).

In [ ]:
# List all amplified directories
response = requests.get(f"{API_BASE}/amplified-directories/")
directories = print_response(response, "LIST AMPLIFIED DIRECTORIES")

if directories:
    print(f"\n✓ Found {directories['total']} amplified director{'y' if directories['total'] == 1 else 'ies'}")
    for dir_info in directories["directories"]:
        print(f"\n  Directory: {dir_info['relativePath']}")
        print(f"  Default Profile: {dir_info['metadata'].get('default_profile', 'N/A')}")
        if "name" in dir_info["metadata"]:
            print(f"  Name: {dir_info['metadata']['name']}")
        print(f"  Created: {dir_info['createdAt']}")

## Create Amplified Directory

Create a new amplified directory with explicit profile and metadata.

In [ ]:
# Create amplified directory with explicit profile
create_request = {
    "relative_path": "projects/myapp",
    "default_profile": "developer-expertise/full",
    "metadata": {
        "name": "My Application",
        "description": "Development project for my app",
        "owner": "developer@example.com",
    },
    "create_marker": True,
}

response = requests.post(f"{API_BASE}/amplified-directories/", json=create_request)
created_dir = print_response(response, "CREATE AMPLIFIED DIRECTORY")

if created_dir:
    print(f"\n✓ Created amplified directory: {created_dir['relativePath']}")
    print(f"  Default Profile: {created_dir['metadata']['default_profile']}")
    print(f"  Name: {created_dir['metadata']['name']}")

## Profile Inheritance

Create a subdirectory without specifying a profile - it will inherit from its parent.

In [ ]:
# Create subdirectory without explicit profile (inherits from parent)
inherit_request = {
    "relative_path": "projects/myapp/frontend",
    "metadata": {"name": "Frontend Module", "description": "React frontend"},
    # Note: No default_profile specified - will inherit from projects/myapp
}

response = requests.post(f"{API_BASE}/amplified-directories/", json=inherit_request)
inherited_dir = print_response(response, "CREATE WITH INHERITANCE")

if inherited_dir:
    parent_profile = created_dir["metadata"]["default_profile"]
    inherited_profile = inherited_dir["metadata"]["default_profile"]
    print("\n✓ Profile Inheritance Verified")
    print(f"  Parent (projects/myapp): {parent_profile}")
    print(f"  Child (projects/myapp/frontend): {inherited_profile}")
    print(f"  Inherited: {parent_profile == inherited_profile}")

## Create Session in Amplified Directory

Sessions can be created in specific amplified directories and will inherit the directory's default profile if not explicitly specified.

In [ ]:
# Create session in amplified directory (inherits profile from directory)
session_request = {
    "amplified_dir": "projects/myapp"
    # Note: No profile_name specified - will use directory's default_profile
}

response = requests.post(f"{API_BASE}/sessions/", json=session_request)
session = print_response(response, "CREATE SESSION IN AMPLIFIED DIR")

if session:
    print("\n✓ Session created in amplified directory")
    print(f"  Session ID: {session['sessionId']}")
    print(f"  Amplified Dir: {session.get('amplifiedDir', 'N/A')}")
    print(f"  Profile: {session['profileName']}")
    print(f"  Inherited from directory: {session['profileName'] == created_dir['metadata']['default_profile']}")

    session_id = session["sessionId"]

## Update Metadata

Update an amplified directory's metadata. Changes are merged with existing metadata.

In [ ]:
# Update metadata (merges with existing)
update_request = {"metadata": {"environment": "staging", "last_deployment": "2025-01-25"}}

response = requests.patch(f"{API_BASE}/amplified-directories/projects/myapp", json=update_request)
updated_dir = print_response(response, "UPDATE METADATA")

if updated_dir:
    print("\n✓ Metadata updated")
    print(f"  All metadata keys: {list(updated_dir['metadata'].keys())}")
    print(f"  New field 'environment': {updated_dir['metadata'].get('environment')}")
    print(f"  Original 'name' preserved: {updated_dir['metadata'].get('name')}")

## Cleanup

Clean up test directories and sessions created in this notebook.

In [ ]:
# Delete session first
if "session_id" in locals():
    response = requests.delete(f"{API_BASE}/sessions/{session_id}")
    if response.status_code == 204:
        print(f"✓ Deleted session: {session_id[:8]}...")

# Delete amplified directories (with marker removal)
test_dirs = ["projects/myapp/frontend", "projects/myapp"]

for dir_path in test_dirs:
    response = requests.delete(f"{API_BASE}/amplified-directories/{dir_path}", params={"remove_marker": True})
    if response.status_code == 204:
        print(f"✓ Deleted directory: {dir_path}")
    elif response.status_code == 404:
        print(f"  Directory not found (already deleted): {dir_path}")
    else:
        print(f"✗ Failed to delete {dir_path}: {response.status_code}")

print("\n✓ Cleanup complete")

## Summary

This notebook demonstrated the amplified directories feature:

### Key Features

1. **Directory Management**
   - Create amplified directories within `AMPLIFIERD_DATA_PATH`
   - List all amplified directories
   - Update metadata
   - Delete directories (with marker removal option)

2. **Profile Inheritance**
   - Root directory auto-amplified on daemon startup
   - Child directories inherit `default_profile` from parent
   - Explicit profile specification overrides inheritance
   - Uses `AMPLIFIERD_DEFAULT_PROFILE` env var for root (default: `foundation/foundation`)

3. **Session Integration**
   - Sessions tied to specific amplified directories
   - Sessions inherit directory's `default_profile` if not specified
   - Sessions default to root directory (`"."`) if no directory specified

4. **Security**
   - All paths validated to prevent directory traversal
   - Rejects absolute paths and `..` components
   - Paths confined to `AMPLIFIERD_DATA_PATH`

### API Endpoints

| Endpoint | Method | Description |
|----------|--------|-------------|
| `/amplified-directories/` | POST | Create amplified directory |
| `/amplified-directories/` | GET | List all amplified directories |
| `/amplified-directories/{path}` | GET | Get specific directory |
| `/amplified-directories/{path}` | PATCH | Update metadata |
| `/amplified-directories/{path}` | DELETE | Delete directory |
| `/sessions/` | POST | Create session (with `amplified_dir` param) |

### Metadata Schema

```json
{
  "default_profile": "foundation/foundation",  // Required
  "name": "Project Name",                       // Optional
  "description": "...",                        // Optional
  // ... other user-defined fields
}
```

### Profile Inheritance Example

```
Root (.)
  └── default_profile: foundation/foundation
      └── projects/myapp
          └── default_profile: developer-expertise/full (explicit)
              └── projects/myapp/frontend
                  └── default_profile: developer-expertise/full (inherited from parent)
```

### Next Steps

- See `07-session-lifecycle.ipynb` for complete session management
- See `03-profile-management.ipynb` for profile system details
- Explore the API documentation at `/docs` for full reference